## AutoGluon-Tabularでのカスタムモデルを用いた機械学習モデルの開発

このノートブックでは、AutoGluon-Tabular を用いて、独自のモデルの持ち込みと組み合わせた高精度な機械学習モデル構築の自動化をご体感頂きます。AutoGluon-Tabular の `AbstractModel` クラスを活用することで、お好みのアルゴリズムの学習やアンサンブルモデルの構築を(半)自動化することができます。

## 準備
必要となるライブラリーをインストールします。

In [ ]:
!pip install --upgrade mxnet
!pip install autogluon

ライブラリをインポートし、必要な設定を行います。

In [ ]:
import time

import autogluon as ag
from autogluon import TabularPrediction as task
from autogluon.task.tabular_prediction.hyperparameter_configs import get_hyperparameter_config
from autogluon.utils.tabular.data.label_cleaner import LabelCleaner
from autogluon.utils.tabular.ml.models.abstract.abstract_model import AbstractModel
from autogluon.utils.tabular.ml.utils import infer_problem_type

# 予測対象となるカラム名を指定します。
label_column = 'class'

# 学習したモデルを保存するディレクトリを指定します。
savedir = 'ag_models/'

### データの取得
このサンプルでは、ある人の年収が50Kを超えるかどうかを二値分類する機械学習モデルを構築します。そのためのデータをダウンロードし、学習用データを準備します。

In [ ]:
train_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
test_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')

train_data = train_data.head(500)
train_data.head()

正解ラベルと特徴量を分離します。

In [ ]:
X_train = train_data.drop(columns=[label_column])
y_train = train_data[label_column]

problem_type = infer_problem_type(y=y_train)

正解ラベルを学習や推論で活用するデータ型へ変換するために、`LabelCleaner` を作成します。str 型だったものが、int 型となっています。これを活用するとデータ型を戻すこともできます。

In [ ]:
label_cleaner = LabelCleaner.construct(problem_type=problem_type, y=y_train)
y_train_clean = label_cleaner.transform(y_train)

テストデータも同様に準備します。

In [ ]:
X_test = test_data.drop(columns=[label_column])
y_test = test_data[label_column]
y_test_clean = label_cleaner.transform(y_test)

### 独自モデルの準備
AutoGluon の組み込みアルゴリズム以外を活用する場合には、`AbstractModel` でラップします。今回の例では、 `scikit-learn` の `Naive Bayse` アルゴリズムを活用します。

In [ ]:
class NaiveBayesModel(AbstractModel):
    def preprocess(self, X):
        cat_columns = X.select_dtypes(['category', 'object']).columns
        X = X.drop(cat_columns, axis=1)
        return super().preprocess(X).fillna(0)

    def _fit(self, X_train, y_train, **kwargs):
        from sklearn.naive_bayes import GaussianNB
        X_train = self.preprocess(X_train)
        self.model = GaussianNB(**self.params)
        self.model.fit(X_train, y_train)

## 独自モデルのみの学習
まずは、先程準備した `NaiveBayesModel` のみを学習させてみましょう。

In [ ]:
naive_bayes_model = NaiveBayesModel(path='AutogluonModels/', name='CustomNaiveBayes', problem_type=problem_type)
naive_bayes_model.fit(X_train=X_train, y_train=y_train_clean)

学習済モデルは保存することができ、使用する場合には、`predictor = task.load(savedir)` のように保存用ディレクトリからロードすることができます。

今回学習させたモデルについて、その精度を評価してみましょう。

In [ ]:
y_pred = naive_bayes_model.predict(X_test)
y_pred_orig = label_cleaner.inverse_transform(y_pred)

score = naive_bayes_model.score(X_test, y_test_clean)
print(f'test score ({naive_bayes_model.eval_metric.name}) = {score}')

In [ ]:
print("Summary of class variable: \n", train_data[label_column].describe())

## AutoGluon の `task` クラスを使った `NaiveBayesModel` の学習

### 学習
今度は、先程と違い Autogluon の `task` クラスを使って学習を行います。これにより、異なるハイパーパラメーターでの学習や、その評価がより簡単に行えます。独自モデルを `task` クラスで活用するために、`custome_hyperparameters` 変数の中で `NaiveBayesModel` を指定しています。今回は `var_smoothing` に異なる3つの値を設定して学習を行い、評価をします。

In [ ]:
custom_hyperparameters = {NaiveBayesModel: [{},{'var_smoothing': 0.00001}, {'var_smoothing': 0.000002}]}  

predictor = task.fit(train_data=train_data, label=label_column, hyperparameters=custom_hyperparameters)  

### 推論

`leaderboard` メソッドを使うと学習の過程で生成されたそれぞれのモデルについて、テストデータ、バリデーションデータでの性能、時間などが表示されます。

In [ ]:
predictor.leaderboard(test_data)

今回学習させたモデルについて、その精度を評価してみましょう。

In [ ]:
y_pred = predictor.predict(test_data)

score = naive_bayes_model.score(X_test, y_test_clean)
print(f'test score ({naive_bayes_model.eval_metric.name}) = {score}')

## AutoGluon の `task` クラスを使い、`NaiveBayesModel` と他のモデルの学習
次に、`NaiveBayesModel`  に加えて、AutoGluon の組み込みアルゴリズムも学習させます。

ハイパーパラメーターの探索を行う場合には、`hp_tune=True` を選択します。また、調整したいハイパーパラメーターの探索領域を辞書形式で渡すことができます。その他、探索試行の回数や、時間制限の目安、探索戦略などを指定することができます。`fit` メソッドの引数に`auto_stack=True` を指定することで、複数層スタッキングを行う事ができます。

In [ ]:
custom_hyperparameters

### 学習

In [ ]:
custom_hyperparameters.update(get_hyperparameter_config('default'))

predictor = task.fit(train_data=train_data, label=label_column, 
                     auto_stack=True, hyperparameters=custom_hyperparameters) 

### 推論

`leaderborad` メソッドや `evaluate_predictions` メソッドを使ってモデルの評価を見てみましょう。

In [ ]:
predictor.leaderboard(test_data)

y_pred = predictor.predict(test_data)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)